<img src="images/logo/eooffshore_banner.png" width="48%" />&nbsp;

<img src="images/logo/seai.png" width="25%" /> <span /> <img src="images/logo/ucd.png" width="7%" />

# NEWA Wind Data for Irish Continental Shelf region

## Introduction

The [New European Wind Atlas (NEWA)](https://www.neweuropeanwindatlas.eu/) provides wind statistics covering onshore Europe, 100km offshore over European seas, and the complete North and Baltic Seas, based on [30 years of mesoscale simulations](https://map.neweuropeanwindatlas.eu/about). Further details may be found in the following publications:

* [Karagali et al. (2018) - New European wind atlas offshore](https://iopscience.iop.org/article/10.1088/1742-6596/1037/5/052007)
* [Sı̄le et al. (2018) - Applying Numerical Weather Prediction Models to the Production of New European Wind Atlas : Sensitivity studies of the wind climate to the planetary boundary layer parametrization](https://doi.org/10.1109/RTUCON.2018.8659881)
* [Hahmann et al. (2020) - The Making of the New European Wind Atlas, Part 1: Model Sensitivity](https://doi.org/10.5194/gmd-13-5053-2020)
* [Dörenkämper et al. (2020) - The Making of the New European Wind Atlas – Part 2: Production and Evaluation](https://doi.org/10.5194/gmd-13-5079-2020)

This notebook provides details of:
1. NEWA wind data products retrieval.
1. The creation of the NEWA Zarr wind stores that are included in the EOOffshore catalog.
1. A brief look at these Zarr stores, including a demonstration of wind speed calculation.

**How to cite:** O'Callaghan, D. and McBreen, S.: Scalable Offshore Wind Analysis With Pangeo, EGU General Assembly 2022, Vienna, Austria, 23–27 May 2022, EGU22-2746, [https://doi.org/10.5194/egusphere-egu22-2746](https://doi.org/10.5194/egusphere-egu22-2746), 2022.

**Note: more extensive usage of the EOOffshore NEWA Zarr stores may be found in the following notebooks:**
* [Offshore Wind in Irish Areas Of Interest](./Offshore_Wind_AOI.ipynb)
* [Comparison of Offshore Wind Speed Extrapolation and Power Density Estimation](./Comparison_Wind_Power.ipynb)

-----------------------------------------------------
## NEWA Wind Data Products

The EOOffshore project uses the mesoscale data products available for selection on the [NEWA Map Layers and Datasets](https://map.neweuropeanwindatlas.eu) website. These contain variables at multiple heights (metres above surface level), which means they are particularly useful for the EOOffshore project due to these heights being similar to typical [wind turbine hub heights](https://www.wind-energy-the-facts.org/hub-height.html). Unlike other data sets used by EOOffshore, they also provide a power density variable in addition to variables related to wind speed and direction. The following data variables are relevant:

| Variable | Unit | Height (metres above sea level) | Description |
| ----------- | ----------- | ----------- | ----------- |
| `WS10` | $m s^{-1}$ | 10 | Wind Speed at 10m |
| `WD10` | degree | 10 | Wind Direction at 10m |
| `WS` | $m s^{-1}$ | 50, 75, 100, 150 | Wind Speed |
| `WD` | degree | 50, 75, 100, 150 | Wind Direction |
| `PD` | $W m^{-2}$ | 50, 75, 100, 150 | Wind Power Density |
| `RHO` | $kg$ $m^{-3}$ | Surface | Air Density |
| $$$$ | $$$$ | $$$$ | $$$$ |

Although both mesoscale and microscale data are available as map layers, plots and data download is only possible for mesoscale outputs. The *Meso Download* tab enables the selection of a time period (day), height, and variables. The *Get script* link will generate a `wget` request for a selected Area Of Interest (AOI), for example;
```bash
wget -O 2015-12-30_Area_1.nc http://opendap.neweuropeanwindatlas.eu:80/opendap/newa/NEWA_MESOSCALE_ATLAS/2015/NEWA-2015-12-30.nc.nc?WS[0:1:47][2:2][646:1:685][244:1:293],time[0:1:47],height[2:2],west_east[244:1:293],south_north[646:1:685],XLON[646:1:685][244:1:293],XLAT[646:1:685][244:1:293],Times[0:1:47],crs
```

This example script will retrieve the selected variables from the corresponding daily [NEWA OPeNDAP data set](http://opendap.neweuropeanwindatlas.eu/opendap/newa/NEWA_MESOSCALE_ATLAS/2015/NEWA-2015-12-30.nc.html). Four AOIs were selected (see below), corresponding daily `wget` commands were constructed for 2009 to 2018, and executed for product retrieval:

|       |  |
| ----------- | ----------- |
| **Observation / Models** | Models |
| **Product type** | Wind Atlas |
| **Processing level** | Level-3 |
| **Data type** | Gridded (latitude/longitude) |
| **Horizontal coverage** | Bounding boxes:<br>[52.15, -7.05, 54.16, -4.75],<br>[51.05, -9.03, 52.50, -6.26],<br>[50.95, -11.14, 51.59, -10.09],<br>[54.75, -10.55, 55.41, -9.40] |
| **Horizontal resolution** | 3 km |
| **Vertical coverage** | [10, 50, 75, 100, 150] metres above surface level |
| **Temporal coverage** | 2009-01-01T00:00:00 to 2018-12-31T23:00:00 |
| **Temporal resolution** | Hourly |
| **Update frequency** | n/a |
| **File format** | NetCDF-4 |
| **Total retrieved products** | 14,611 |
| **Total products size** | 27G |

-----------------------------------------------------
## NEWA Wind Zarr Stores

For each AOI data set:
* the retrieved NetCDF products were loaded using [`xarray.open_mfdataset()`](https://xarray.pydata.org/en/stable/generated/xarray.open_mfdataset.html), combined by their grid coordinates and concatenated along the `time` dimension. 
* the data set was chunked in time and persisted to a single, chunked, compressed [Zarr](https://zarr.readthedocs.io/en/stable/) store (16G), which is a cloud-optimised format suitable for multi-dimensional arrays. A `time` chunk size was specified that resulted in a low number of `time` chunks, as this approach is more suitable for subsequent processing of variables over time for AOIs.

Zarr stores were created for four AOIs:
* Irish Sea (8.7 GB)
* Celtic Sea (7.9 GB)
* Area surrounding the [Irish Weather Buoy Network (IWB)](http://www.marine.ie/Home/site-area/data-services/real-time-observations/irish-weather-buoy-network-imos) M3 buoy (1.4G)
* Area surrounding the IWB M4 buoy (1.4G)

As requested by the [NEWA Terms of use](https://map.neweuropeanwindatlas.eu/about), the following attribution is declared:
* Data [2009 - 2018] obtained from the New European Wind Atlas (NEWA), a free, web-based application developed, owned and operated by the NEWA Consortium. For additional information see www.neweuropeanwindatlas.eu.

-----------------------------------------------
## NEWA in EOOffshore Catalog

Imports required for subsequent processing

In [1]:
from dataclasses import dataclass, field
from intake import Catalog, open_catalog
import json
import xarray as xr

### Open the catalog and view the NEWA metadata

All EOOffshore data sets, including the NEWA Zarr stores described above, are accessible using the EOOffshore [Intake](https://intake.readthedocs.io/en/latest/) catalog. Each [catalog](https://intake.readthedocs.io/en/latest/catalog.html) entry provides a description and metadata associated with the corresponding data set, defined in a [YAML configuration file](https://intake.readthedocs.io/en/latest/catalog.html#yaml-format). The EOOffshore catalog configuration was originally influenced by the [Pangeo Cloud Data Store atmosphere.yaml catalog configuration](https://github.com/pangeo-data/pangeo-datastore/blob/master/intake-catalogs/atmosphere.yaml). 

In [2]:
catalog = open_catalog('data/intake-catalogs/eooffshore_ics.yaml')

View the Irish Sea NEWA Zarr store metadata

In [3]:
catalog.eooffshore_ics_newa_irishsea

eooffshore_ics_newa_irishsea:
  args:
    storage_options: null
    urlpath: /data/eo/zarr/newa/eooffshore_ics_newa_irishsea.zarr
  description: EOOffshore Project 2009 - 2018 Concatenated NEWA wind variable products
    for Irish Sea area within the Irish Continental Shelf. Data [2009 - 2018] obtained
    from the New European Wind Atlas (NEWA), a free, web-based application developed,
    owned and operated by the NEWA Consortium. For additional information see www.neweuropeanwindatlas.eu.'
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: /opt/eooffshore/notebooks/datasets/data/intake-catalogs/
    tags:
    - atmosphere
    - wind
    - newa
    - iwb
    - ocean
    title: NEWA products for the Irish Sea area of interest (2009 - 2018)
    url: https://map.neweuropeanwindatlas.eu/


View the Celtic Sea NEWA Zarr store metadata

In [4]:
catalog.eooffshore_ics_newa_celticsea

eooffshore_ics_newa_celticsea:
  args:
    storage_options: null
    urlpath: /data/eo/zarr/newa/eooffshore_ics_newa_celticsea.zarr
  description: EOOffshore Project 2009 - 2018 Concatenated NEWA wind variable products
    for North Celtic Sea area within the Irish Continental Shelf. Data [2009 - 2018]
    obtained from the New European Wind Atlas (NEWA), a free, web-based application
    developed, owned and operated by the NEWA Consortium. For additional information
    see www.neweuropeanwindatlas.eu.'
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: /opt/eooffshore/notebooks/datasets/data/intake-catalogs/
    tags:
    - atmosphere
    - wind
    - newa
    - iwb
    - ocean
    title: NEWA products for the Celtic Sea area of interest (2009 - 2018)
    url: https://map.neweuropeanwindatlas.eu/


View the [Irish Weather Buoy Network - M3 buoy](http://www.marine.ie/Home/site-area/data-services/real-time-observations/irish-weather-buoy-network-imos) NEWA Zarr store metadata

In [5]:
catalog.eooffshore_ics_newa_m3

eooffshore_ics_newa_m3:
  args:
    storage_options: null
    urlpath: /data/eo/zarr/newa/eooffshore_ics_newa_m3.zarr
  description: EOOffshore Project 2009 - 2018 Concatenated NEWA wind variable products
    for surrounding the area surrounding the Irish Weather Buoy Network (IWB) M3 buoy
    within the Irish Continental Shelf. Data [2009 - 2018] obtained from the New European
    Wind Atlas (NEWA), a free, web-based application developed, owned and operated
    by the NEWA Consortium. For additional information see www.neweuropeanwindatlas.eu.'
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: /opt/eooffshore/notebooks/datasets/data/intake-catalogs/
    tags:
    - atmosphere
    - wind
    - newa
    - iwb
    - ocean
    title: NEWA products for the IWB M3 buoy area (2009 - 2018)
    url: https://map.neweuropeanwindatlas.eu/


View the [Irish Weather Buoy Network - M4 buoy](http://www.marine.ie/Home/site-area/data-services/real-time-observations/irish-weather-buoy-network-imos) NEWA Zarr store metadata

In [6]:
catalog.eooffshore_ics_newa_m4

eooffshore_ics_newa_m4:
  args:
    storage_options: null
    urlpath: /data/eo/zarr/newa/eooffshore_ics_newa_m4.zarr
  description: EOOffshore Project 2009 - 2018 Concatenated NEWA wind variable products
    for surrounding the area surrounding the Irish Weather Buoy Network (IWB) M4 buoy
    within the Irish Continental Shelf. Data [2009 - 2018] obtained from the New European
    Wind Atlas (NEWA), a free, web-based application developed, owned and operated
    by the NEWA Consortium. For additional information see www.neweuropeanwindatlas.eu.'
  driver: intake_xarray.xzarr.ZarrSource
  metadata:
    catalog_dir: /opt/eooffshore/notebooks/datasets/data/intake-catalogs/
    tags:
    - atmosphere
    - wind
    - newa
    - iwb
    - ocean
    title: NEWA products for the IWB M4 buoy area (2009 - 2018)
    url: https://map.neweuropeanwindatlas.eu/


----------------------------------------------------------------
### Load each catalog NEWA Zarr store

Intake catalog entries typically specify a [driver](https://intake.readthedocs.io/en/latest/catalog.html#driver-selection) to be used when loading the corresponding data set. The NEWA entries specify [`intake_xarray.xzarr.ZarrSource`](https://intake-xarray.readthedocs.io/en/latest/api.html#intake_xarray.xzarr.ZarrSource), a driver implementation provided by the [intake-xarray](https://intake-xarray.readthedocs.io/) library. This enables NetCDF and Zarr data sets to be loaded using [xarray](https://docs.xarray.dev/en/stable/index.html), a library for processing N-D labeled arrays and datasets. As xarray labels take the form of dimensions, coordinates and attributes on top of [NumPy](https://numpy.org/)-like arrays, it is particularly suited to data sets such as NEWA whose variables feature latitude/longitude grid coordinates.

This intake driver will load the associated dataset into an [`xarray.Dataset`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html). To enable support for potentially large data sets, the [`to_dask()`](https://intake.readthedocs.io/en/latest/quickstart.html?#working-with-dask) function is used to load the underlying variable arrays with [Dask](https://docs.dask.org/en/latest/), a parallel, out-of-core computing library. The [`ZarrSource`](https://intake-xarray.readthedocs.io/en/latest/api.html#intake_xarray.xzarr.ZarrSource) implementation will load the data set variables into [Dask arrays](https://docs.dask.org/en/latest/array.html), which will be loaded and processed in parallel as [chunks](https://docs.dask.org/en/latest/array.html) during subsequent computation. As discussed above, variable chunk sizes may be specified during Zarr store creation.

This utility class is used for Zarr loading and preparation. The 10m NEWA wind variables (`WS10` and `WD10`) are added to the corresponding wind variables containing data at multiple heights (`WS` and `WD`), and the `height` dimension is reindexed to include the 10m data.

In [7]:
@dataclass
class EooffshoreDataset:
    name: str
    catalog: Catalog = field(repr=False, hash=False, compare=False)
    dataset_id: str
    dataset: xr.Dataset = field(init=False, hash=False, compare=False, repr=False)

    def __post_init__(self):
        """Load the dataset from the Intake catalog"""
        ds = self.catalog[self.dataset_id].to_dask()
        
        # Enable consistent variable naming
        ds = ds.rename({'WS': 'wind_speed', 
                        'WD': 'wind_direction', 
                        'PD': 'power_density',
                        'XLAT': 'latitude',
                        'XLON': 'longitude'}
                      )
        # Not required
        ds = ds.drop(['south_north', 'west_east'])

        # Add provided 10m wind variable data to corresponding variable with a height dimension
        wind_variables = {}
        for variable_name_10m, variable_name in {'WS10': 'wind_speed', 'WD10': 'wind_direction'}.items(): 
            wind_variables[variable_name] = xr.concat((ds[variable_name], ds[variable_name_10m].assign_coords(height=10).expand_dims(['height'])), dim='height').sortby('height')
        ds = ds.reindex({'height': wind_variables['wind_speed'].height}).chunk({'height':1})
        for variable_name, wind_variable in wind_variables.items():
            ds[variable_name] = wind_variable
        
        self.dataset = ds

Here, each NEWA store is loaded into an [`xarray.Dataset`](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html):

* The `wind_speed`, `wind_direction` and `power_density` variables have associated coordinate dimensions:
  * `height`
  * `west_east` (longitude) and `south_north` (latitude) - the corresponding coordinate grid
  * `time` - hourly
* A low number of `time` chunks have been specified, to support subsequent computation across time for smaller AOI grid coordinates.

In [8]:
newa_irishsea = EooffshoreDataset(name='NEWA Irish Sea', catalog=catalog, dataset_id='eooffshore_ics_newa_irishsea')
newa_celticsea = EooffshoreDataset(name='NEWA Celtic Sea', catalog=catalog, dataset_id='eooffshore_ics_newa_celticsea')
newa_m3 = EooffshoreDataset(name='NEWA M3', catalog=catalog, dataset_id='eooffshore_ics_newa_m3')
newa_m4 = EooffshoreDataset(name='NEWA M4', catalog=catalog, dataset_id='eooffshore_ics_newa_m4')

In [9]:
newa_irishsea.dataset

<xarray.Dataset>
Dimensions:         (height: 5, time: 87648, south_north: 67, west_east: 35)
Coordinates:
  * height          (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude        (south_north, west_east) float32 dask.array<chunksize=(67, 35), meta=np.ndarray>
    longitude       (south_north, west_east) float32 dask.array<chunksize=(67, 35), meta=np.ndarray>
  * time            (time) datetime64[ns] 2009-01-01 ... 2018-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Data variables:
    power_density   (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 67, 35), meta=np.ndarray>
    RHO             (time, south_north, west_east) float32 dask.array<chunksize=(40000, 67, 35), meta=np.ndarray>
    Times           (time) |S20 dask.array<chunksize=(40000,), meta=np.ndarray>
    wind_direction  (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 67, 35), meta=np.ndarray>
    WD10            (time, south_north, west_east) float32 dask.array<chunksize=(40000, 67, 35), meta=np.ndarray>
    wind_speed      (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 67, 35), meta=np.ndarray>
    WS10            (time, south_north, west_east) float32 dask.array<chunksize=(40000, 67, 35), meta=np.ndarray>
    crs             (time) |S1 dask.array<chunksize=(40000,), meta=np.ndarray>
Attributes: (12/116)
    ADAPT_DT_MAX:                    15.0
    ADAPT_DT_MIN:                    3.0
    ADAPT_DT_START:                  13.0
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    ...                              ...
    WEST-EAST_GRID_DIMENSION:        Attribute edlided: Unsupported attribute...
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    eooffshore_zarr_creation_time:   2022-05-13T15:25:40Z
    eooffshore_zarr_details:         EOOffshore Project: Concatenated NEWA wi...
    history:                         2021-10-27 13:36:24 GMT Hyrax-1.16.0 htt...

In [10]:
newa_celticsea.dataset

<xarray.Dataset>
Dimensions:         (height: 5, time: 87648, south_north: 40, west_east: 54)
Coordinates:
  * height          (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude        (south_north, west_east) float32 dask.array<chunksize=(40, 54), meta=np.ndarray>
    longitude       (south_north, west_east) float32 dask.array<chunksize=(40, 54), meta=np.ndarray>
  * time            (time) datetime64[ns] 2009-01-01 ... 2018-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Data variables:
    power_density   (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 40, 54), meta=np.ndarray>
    RHO             (time, south_north, west_east) float32 dask.array<chunksize=(40000, 40, 54), meta=np.ndarray>
    Times           (time) |S20 dask.array<chunksize=(40000,), meta=np.ndarray>
    wind_direction  (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 40, 54), meta=np.ndarray>
    WD10            (time, south_north, west_east) float32 dask.array<chunksize=(40000, 40, 54), meta=np.ndarray>
    wind_speed      (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 40, 54), meta=np.ndarray>
    WS10            (time, south_north, west_east) float32 dask.array<chunksize=(40000, 40, 54), meta=np.ndarray>
    crs             (time) |S1 dask.array<chunksize=(40000,), meta=np.ndarray>
Attributes: (12/116)
    ADAPT_DT_MAX:                    15.0
    ADAPT_DT_MIN:                    3.0
    ADAPT_DT_START:                  13.0
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    ...                              ...
    WEST-EAST_GRID_DIMENSION:        Attribute edlided: Unsupported attribute...
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    eooffshore_zarr_creation_time:   2022-05-13T15:11:52Z
    eooffshore_zarr_details:         EOOffshore Project: Concatenated NEWA wi...
    history:                         2021-10-27 13:36:09 GMT Hyrax-1.16.0 htt...

In [11]:
newa_m3.dataset

<xarray.Dataset>
Dimensions:         (height: 5, time: 87648, south_north: 19, west_east: 20)
Coordinates:
  * height          (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude        (south_north, west_east) float32 dask.array<chunksize=(19, 20), meta=np.ndarray>
    longitude       (south_north, west_east) float32 dask.array<chunksize=(19, 20), meta=np.ndarray>
  * time            (time) datetime64[ns] 2009-01-01 ... 2018-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Data variables:
    power_density   (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 19, 20), meta=np.ndarray>
    RHO             (time, south_north, west_east) float32 dask.array<chunksize=(40000, 19, 20), meta=np.ndarray>
    Times           (time) |S20 dask.array<chunksize=(40000,), meta=np.ndarray>
    wind_direction  (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 19, 20), meta=np.ndarray>
    WD10            (time, south_north, west_east) float32 dask.array<chunksize=(40000, 19, 20), meta=np.ndarray>
    wind_speed      (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 19, 20), meta=np.ndarray>
    WS10            (time, south_north, west_east) float32 dask.array<chunksize=(40000, 19, 20), meta=np.ndarray>
    crs             (time) |S1 dask.array<chunksize=(40000,), meta=np.ndarray>
Attributes: (12/116)
    ADAPT_DT_MAX:                    15.0
    ADAPT_DT_MIN:                    3.0
    ADAPT_DT_START:                  13.0
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    ...                              ...
    WEST-EAST_GRID_DIMENSION:        Attribute edlided: Unsupported attribute...
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    eooffshore_zarr_creation_time:   2022-05-13T15:39:49Z
    eooffshore_zarr_details:         EOOffshore Project: Concatenated NEWA wi...
    history:                         2021-10-27 13:38:03 GMT Hyrax-1.16.0 htt...

In [12]:
newa_m4.dataset

<xarray.Dataset>
Dimensions:         (height: 5, time: 87648, south_north: 20, west_east: 20)
Coordinates:
  * height          (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude        (south_north, west_east) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
    longitude       (south_north, west_east) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
  * time            (time) datetime64[ns] 2009-01-01 ... 2018-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Data variables:
    power_density   (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 20, 20), meta=np.ndarray>
    RHO             (time, south_north, west_east) float32 dask.array<chunksize=(40000, 20, 20), meta=np.ndarray>
    Times           (time) |S20 dask.array<chunksize=(40000,), meta=np.ndarray>
    wind_direction  (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 20, 20), meta=np.ndarray>
    WD10            (time, south_north, west_east) float32 dask.array<chunksize=(40000, 20, 20), meta=np.ndarray>
    wind_speed      (time, height, south_north, west_east) float32 dask.array<chunksize=(40000, 1, 20, 20), meta=np.ndarray>
    WS10            (time, south_north, west_east) float32 dask.array<chunksize=(40000, 20, 20), meta=np.ndarray>
    crs             (time) |S1 dask.array<chunksize=(40000,), meta=np.ndarray>
Attributes: (12/116)
    ADAPT_DT_MAX:                    15.0
    ADAPT_DT_MIN:                    3.0
    ADAPT_DT_START:                  13.0
    AER_ANGEXP_OPT:                  1
    AER_ANGEXP_VAL:                  1.2999999523162842
    AER_AOD550_OPT:                  1
    ...                              ...
    WEST-EAST_GRID_DIMENSION:        Attribute edlided: Unsupported attribute...
    W_DAMPING:                       1
    YSU_TOPDOWN_PBLMIX:              0
    eooffshore_zarr_creation_time:   2022-05-13T15:52:39Z
    eooffshore_zarr_details:         EOOffshore Project: Concatenated NEWA wi...
    history:                         2021-10-27 13:38:07 GMT Hyrax-1.16.0 htt...

----------------------------------------------------------------
### NEWA wind speed (2009 - 2018)
Each variable in the NEWA data sets, for example, wind speed, is loaded into an [`xarray.DataArray`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html):
* The arrays are transposed here to produce more intuitive `time` chunk visualisations. This is not required for computation.

In [13]:
newa_irishsea.dataset.wind_speed.transpose('height', 'time', 'south_north', 'west_east')

<xarray.DataArray 'wind_speed' (height: 5, time: 87648, south_north: 67,
                                west_east: 35)>
dask.array<transpose, shape=(5, 87648, 67, 35), dtype=float32, chunksize=(1, 40000, 67, 35), chunktype=numpy.ndarray>
Coordinates:
  * height     (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude   (south_north, west_east) float32 dask.array<chunksize=(67, 35), meta=np.ndarray>
    longitude  (south_north, west_east) float32 dask.array<chunksize=(67, 35), meta=np.ndarray>
  * time       (time) datetime64[ns] 2009-01-01 ... 2018-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    grid_mapping:   crs
    long_name:      Wind Speed
    standard_name:  wind_speed
    units:          m s-1

In [14]:
newa_celticsea.dataset.wind_speed.transpose('height', 'time', 'south_north', 'west_east')

<xarray.DataArray 'wind_speed' (height: 5, time: 87648, south_north: 40,
                                west_east: 54)>
dask.array<transpose, shape=(5, 87648, 40, 54), dtype=float32, chunksize=(1, 40000, 40, 54), chunktype=numpy.ndarray>
Coordinates:
  * height     (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude   (south_north, west_east) float32 dask.array<chunksize=(40, 54), meta=np.ndarray>
    longitude  (south_north, west_east) float32 dask.array<chunksize=(40, 54), meta=np.ndarray>
  * time       (time) datetime64[ns] 2009-01-01 ... 2018-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    grid_mapping:   crs
    long_name:      Wind Speed
    standard_name:  wind_speed
    units:          m s-1

In [15]:
newa_m3.dataset.wind_speed.transpose('height', 'time', 'south_north', 'west_east')

<xarray.DataArray 'wind_speed' (height: 5, time: 87648, south_north: 19,
                                west_east: 20)>
dask.array<transpose, shape=(5, 87648, 19, 20), dtype=float32, chunksize=(1, 40000, 19, 20), chunktype=numpy.ndarray>
Coordinates:
  * height     (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude   (south_north, west_east) float32 dask.array<chunksize=(19, 20), meta=np.ndarray>
    longitude  (south_north, west_east) float32 dask.array<chunksize=(19, 20), meta=np.ndarray>
  * time       (time) datetime64[ns] 2009-01-01 ... 2018-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    grid_mapping:   crs
    long_name:      Wind Speed
    standard_name:  wind_speed
    units:          m s-1

In [16]:
newa_m4.dataset.wind_speed.transpose('height', 'time', 'south_north', 'west_east')

<xarray.DataArray 'wind_speed' (height: 5, time: 87648, south_north: 20,
                                west_east: 20)>
dask.array<transpose, shape=(5, 87648, 20, 20), dtype=float32, chunksize=(1, 40000, 20, 20), chunktype=numpy.ndarray>
Coordinates:
  * height     (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude   (south_north, west_east) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
    longitude  (south_north, west_east) float32 dask.array<chunksize=(20, 20), meta=np.ndarray>
  * time       (time) datetime64[ns] 2009-01-01 ... 2018-12-31T23:00:00
Dimensions without coordinates: south_north, west_east
Attributes:
    grid_mapping:   crs
    long_name:      Wind Speed
    standard_name:  wind_speed
    units:          m s-1

#### Calculate mean wind speed over `time` dimension for all heights at AOI grid coordinates

Using Dask, the data set loading process is lazy, where no data is loaded inititally. Instead, data loading is [delayed until execution time, where variables will be loaded and processed in parallel according to the corresponding chunks specification](https://tutorial.dask.org/01x_lazy.html). Dask arrays implement a subset of the NumPy [`ndarray`](https://numpy.org/doc/stable/reference/arrays.ndarray.html) interface using blocked algorithms, and the original variable arrays will be split into smaller chunk arrays, enabling computation on arrays larger than memory using all available cores. The blocked algorithms are coordinated using [Dask graphs](https://docs.dask.org/en/stable/graphs.html).

To perform some analysis at known AOI latitude/longitude coordinates, the [`xarray.DataArray.sel(..., method='nearest')`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.sel.html) function may be used to select a subset of the data array (or data set) at coordinates nearest to the specified parameters. 
Here, mean wind speed over the `time` dimension is determined for the specified coordinates, where Dask graph execution is triggered by calling [`compute()`](https://docs.dask.org/en/stable/api.html#dask.compute).  The resulting variable values will be contained in a NumPy `ndarray`.
* **Note:** [`xarray.DataArray.sel(..., method='nearest')`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.sel.html) does not support selection from two-dimensional curvilinear grids as used in the NEWA data sets. Consequently, [`xarray.DataArray.isel()`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.isel.html) is used here with the `x` (longitude) and `y` (latitude) dimensions for demonstration. A solution enabling two-dimensional grid selection is described in the [Offshore Wind in Irish Areas Of Interest](./Offshore_Wind_AOI.ipynb) notebook.

Graph execution is managed by a [task scheduler](https://docs.dask.org/en/stable/scheduling.html). The default scheduler (used for executing this notebook) executes computations with [local threads](https://docs.dask.org/en/stable/scheduling.html#local-threads). However, execution may also be performed on a [distributed cluster](https://docs.dask.org/en/stable/scheduling.html#dask-distributed-local) **without any change to the `xarray` code used here**.

In [17]:
newa_irishsea.dataset.wind_speed.isel(west_east=0, south_north=0).mean(dim='time').compute()

<xarray.DataArray 'wind_speed' (height: 5)>
array([8.081513, 9.090129, 9.398023, 9.641414, 9.994202], dtype=float32)
Coordinates:
  * height     (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude   float32 52.15
    longitude  float32 -6.238

In [18]:
newa_celticsea.dataset.wind_speed.isel(west_east=0, south_north=0).mean(dim='time').compute()

<xarray.DataArray 'wind_speed' (height: 5)>
array([ 8.580611,  9.392164,  9.646666,  9.849321, 10.153699],
      dtype=float32)
Coordinates:
  * height     (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude   float32 51.05
    longitude  float32 -8.522

In [19]:
newa_m3.dataset.wind_speed.isel(west_east=0, south_north=0).mean(dim='time').compute()

<xarray.DataArray 'wind_speed' (height: 5)>
array([ 9.164568,  9.935695, 10.173062, 10.357634, 10.630132],
      dtype=float32)
Coordinates:
  * height     (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude   float32 50.95
    longitude  float32 -10.89

In [20]:
newa_m4.dataset.wind_speed.isel(west_east=0, south_north=0).mean(dim='time').compute()

<xarray.DataArray 'wind_speed' (height: 5)>
array([ 9.485068, 10.272412, 10.511473, 10.694211, 10.965795],
      dtype=float32)
Coordinates:
  * height     (height) float64 10.0 50.0 75.0 100.0 150.0
    latitude   float32 54.75
    longitude  float32 -10.26